In [15]:
import pandas as pd
import time
import numpy
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
%matplotlib inline

In [259]:
df_for_ml = pd.read_pickle('df_for_ml.pkl')

Examine some other regressors to see whether linear regression results can be improved upon. We'll use polynomial features some of the time.

In [31]:
from sklearn import tree

In [254]:
X = df_for_ml.drop(['boughts', 'loves', 'comments', 'impressions', 'clicks', 'unloves'], axis=1)
y = df_for_ml['loves']
XTrain, XTest, yTrain, yTest = train_test_split(X, y, random_state=1, test_size=0.2)


In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [54]:
parameters = {'max_depth':[2,4,6, 8, 10, 12, 15, 17, 20, 50],"min_samples_split":[200, 300, 350, 400, 500, 600, 700, 800] }
dt = tree.DecisionTreeRegressor()

clf = GridSearchCV(dt, parameters,scoring=make_scorer(metrics.r2_score, greater_is_better=True))
clf.fit(XTrain, yTrain)

print("best parameter set is:",clf.best_params_," and its score was",clf.best_score_)

best parameter set is: {'max_depth': 15, 'min_samples_split': 400}  and its score was 0.1609624890969235


In [17]:
from sklearn.ensemble import RandomForestRegressor

In [270]:
def print_train_test_metrics():
    print('Accuracy on training data= ', metrics.r2_score(y_true = yTrain, y_pred = y_pred_train))
    print('Accuracy on test data= ', metrics.r2_score(y_true = yTest, y_pred = y_pred))


In [272]:
forest =  RandomForestRegressor(bootstrap=True, n_estimators=300, random_state=0)

trained_forest = forest.fit(XTrain, yTrain) 

y_pred_train = trained_forest.predict(XTrain)

y_pred = trained_forest.predict(XTest)

print_train_test_metrics()


Accuracy on training data=  0.8939916729959491
Accuracy on test data=  0.18875400738838766


In [271]:
forest =  RandomForestRegressor(bootstrap=True, n_estimators=400, random_state=0)

trained_forest = forest.fit(XTrain, yTrain) 

y_pred_train = trained_forest.predict(XTrain)

y_pred = trained_forest.predict(XTest)

print_train_test_metrics()

Accuracy on training data=  0.8942715517140571
Accuracy on test data=  0.18904849332717122


In [273]:
featuresDF=pd.DataFrame({"features":X.columns,"weights":trained_forest.feature_importances_})
featuresDF.sort_values("weights",ascending=False)

,features,weights
7,desc_word_count,0.228952
3,user.rep,0.094523
1,user.deals_count,0.088267
17,total_price_in_usd,0.077549
15,price_in_usd,0.076357
12,desc_sentiment_compound,0.071085
18,title_word_count,0.062667
10,desc_sentiment_neu,0.056087
11,desc_sentiment_pos,0.050996
2,user.loves_count,0.046701


In [65]:
from sklearn.linear_model import BayesianRidge

In [226]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model, pipeline

In [277]:
br = BayesianRidge(compute_score=True)
br.fit(X, y)
y_pred_train = br.predict(XTrain)
y_pred = br.predict(XTest)
print_train_test_metrics()


Accuracy on training data=  0.17602439550187188
Accuracy on test data=  0.15794252275796283


In [280]:
def print_short_metrics():
    print(f"mse: {metrics.mean_squared_error(yTest,yPred )} ")
    print(f"r^2: {metrics.r2_score(yTest,yPred )} ")

In [286]:
model = pipeline.make_pipeline(PolynomialFeatures(degree=2), br)
model.fit(XTrain, yTrain)
yPred = model.predict(XTest)
print_short_metrics()

mse: 17.731900396223214 
r^2: 0.21778368470807874 


In [281]:
rcv = linear_model.RidgeCV(alphas=[0.1, 0.5, 1.0, 2.0, 5.0, 10.0], cv=15, fit_intercept=True, scoring=None, normalize=False)
rcv.fit(XTrain, yTrain)
yPred = rcv.predict(XTest)
print_short_metrics()

mse: 19.092468817114796 
r^2: 0.1577642399158533 


In [282]:
model = pipeline.make_pipeline(PolynomialFeatures(degree=2), linear_model.RidgeCV(alphas=[0.1, 0.5, 1.0, 2.0, 5.0, 10.0], cv=15, fit_intercept=True, scoring=None, normalize=False))
model.fit(XTrain, yTrain)
yPred = model.predict(XTest)
print_short_metrics()

/Users/b/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=9.80547e-23): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/b/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.03138e-22): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/b/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=9.87025e-23): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/b/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=8.04553e-23): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/b/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=

/Users/b/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=6.03112e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/b/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=5.93739e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/b/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=5.49624e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/b/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=5.21352e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/b/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=

mse: 17.641244689747253 
r^2: 0.22178282586583742 


/Users/b/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.99873e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


In [284]:
ridge = linear_model.Ridge()
ridge.fit(XTrain, yTrain)
yPred = ridge.predict(XTest)
print_short_metrics()

mse: 19.09090265391919 
r^2: 0.15783332886721424 


In [285]:
model = pipeline.make_pipeline(PolynomialFeatures(degree=2), linear_model.Ridge())
model.fit(XTrain, yTrain)
yPred = model.predict(XTest)
print_short_metrics()

mse: 17.645858468272337 
r^2: 0.22157929591378367 


/Users/b/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.12212e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


In [242]:
model = pipeline.make_pipeline(PolynomialFeatures(degree=2), RandomForestRegressor(bootstrap=True, n_estimators=400, random_state=0))
model.fit(XTrain, yTrain)
yPred = model.predict(XTest)
print_short_metrics()

mse: 0.0007324507228685822 
r^2: 0.23720876341437325 
